In [1]:
from chords_and_notes_dict import * 
import threading 
import tkinter as tk
from PIL import ImageTk, Image
import mido
import random 
import os
from functools import partial
import pickle
import time
from datetime import datetime
import copy



In [ ]:
def open_port(input: str, output: str): 
    """ 
    Input_port: MIDI controller to send MIDI messages 
    Output_port: Virtual port opened in Ableton to convey the MIDI message in sound
    """

    global input_port, output_port

    #if input_port and output_port are AVAILABLE
    try: 
        input_port = mido.open_input(input)
        output_port = mido.open_output(output)

    #if input_port and output_port are NOT AVAILABLE    
    except Exception:
        close_port(input_port, output_port)
        input_port = mido.open_input(input)
        output_port = mido.open_output(output)
    
    
    return input_port, output_port 


def close_port(input_port, output_port):
    """ Close input and output ports"""
    input_port.close()
    output_port.close()


